In [1]:
import arrow
import socket
from sqlalchemy.orm import Session
from tqdm import tqdm

from april import Evaluator
from april.anomalydetection import *
from april.database import EventLog
from april.database import Model
from april.database import get_engine
from april.dataset import Dataset
from april.fs import DATE_FORMAT
from april.fs import get_event_log_files

This is the train method that can be called in parallel.

In [2]:
def fit_and_save(dataset_name, ad, ad_kwargs=None, fit_kwargs=None):
    if ad_kwargs is None:
        ad_kwargs = {}
    if fit_kwargs is None:
        fit_kwargs = {}

    # Save start time
    start_time = arrow.now()

    # Dataset
    dataset = Dataset(dataset_name)

    # AD
    ad = ad(**ad_kwargs)

    # Train and save
    ad.fit(dataset, **fit_kwargs)
    file_name = f'{dataset_name}_{ad.abbreviation}_{start_time.format(DATE_FORMAT)}'
    model_file = ad.save(file_name)

    # Save end time
    end_time = arrow.now()

    # Cache result
    #print(model_file.str_path)
    Evaluator(model_file.str_path).cache_result()

    # Calculate training time in seconds
    training_time = (end_time - start_time).total_seconds()

    # Write to database
    engine = get_engine()
    session = Session(engine)

    session.add(Model(creation_date=end_time.datetime,
                      algorithm=ad.name,
                      training_duration=training_time,
                      file_name=model_file.file,
                      training_event_log_id=EventLog.get_id_by_name(dataset_name),
                      training_host=socket.gethostname(),
                      hyperparameters=str(dict(**ad_kwargs, **fit_kwargs))))
    session.commit()
    session.close()

    if isinstance(ad, NNAnomalyDetector):
        from keras.backend import clear_session
        clear_session()

Now, we can run the training for the anomaly detection methods.

In [ ]:
datasets = sorted([e.name for e in get_event_log_files() if e.p == 0.3])
ads = [
    #dict(ad=RandomAnomalyDetector),
    #dict(ad=TStidePlus, ad_kwargs=dict(k=2)),
    dict(ad=OneClassSVM),
    #dict(ad=LikelihoodPlusAnomalyDetector),
    #dict(ad=BoehmerLikelihoodAnomalyDetector),
    dict(ad=NaiveAnomalyDetector),
    #dict(ad=NaivePlusAnomalyDetector),
    dict(ad=SamplingAnomalyDetector),
    dict(ad=DAE, fit_kwargs=dict(epochs=50, batch_size=500)),
    dict(ad=BINetv1, fit_kwargs=dict(epochs=20, batch_size=500)),
    #dict(ad=BINetv2, fit_kwargs=dict(epochs=20, batch_size=500)),
    #dict(ad=BINetv3, fit_kwargs=dict(epochs=20, batch_size=500))
]
for ad in ads:
    [fit_and_save(d, **ad) for d in tqdm(datasets, desc=ad['ad'].name)]

OC-SVM:   0%|                                                                                   | 0/10 [00:00<?, ?it/s]C:\Users\Jonghyeon\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Jonghyeon\Anaconda3\lib\site-packages\arrow\factory.py:249: ArrowParseWarning: The .get() parsing method with a format string will parse more strictly in version 0.15.0.See https://github.com/crsmithdev/arrow/issues/612 for more details.
  ArrowParseWarning,
C:\Users\Jonghyeon\Anaconda3\lib\site-packages\arrow\factory.py:249: ArrowParseWarning: The .get() parsing method with a format string will parse more strictly in version 0.15.0.See https://github.com/crsmithdev/arrow/issues/612 for more details.
  ArrowParseWarning,
C:\Users\Jonghyeon\Anaconda3\lib\s

C:\Users\Jonghyeon\Anaconda3\lib\site-packages\arrow\factory.py:249: ArrowParseWarning: The .get() parsing method with a format string will parse more strictly in version 0.15.0.See https://github.com/crsmithdev/arrow/issues/612 for more details.
  ArrowParseWarning,
OC-SVM:  70%|████████████████████████████████████████████████████▌                      | 7/10 [00:13<00:05,  1.68s/it]C:\Users\Jonghyeon\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Jonghyeon\Anaconda3\lib\site-packages\arrow\factory.py:249: ArrowParseWarning: The .get() parsing method with a format string will parse more strictly in version 0.15.0.See https://github.com/crsmithdev/arrow/issues/612 for more details.
  ArrowParseWarning,
C:\Users\Jonghyeon\Anaconda3\lib\s

C:\Users\Jonghyeon\Anaconda3\lib\site-packages\arrow\factory.py:249: ArrowParseWarning: The .get() parsing method with a format string will parse more strictly in version 0.15.0.See https://github.com/crsmithdev/arrow/issues/612 for more details.
  ArrowParseWarning,
Naive:  50%|██████████████████████████████████████                                      | 5/10 [00:00<00:00,  8.24it/s]C:\Users\Jonghyeon\Anaconda3\lib\site-packages\arrow\factory.py:249: ArrowParseWarning: The .get() parsing method with a format string will parse more strictly in version 0.15.0.See https://github.com/crsmithdev/arrow/issues/612 for more details.
  ArrowParseWarning,
C:\Users\Jonghyeon\Anaconda3\lib\site-packages\arrow\factory.py:249: ArrowParseWarning: The .get() parsing method with a format string will parse more strictly in version 0.15.0.See https://github.com/crsmithdev/arrow/issues/612 for more details.
  ArrowParseWarning,
C:\Users\Jonghyeon\Anaconda3\lib\site-packages\arrow\factory.py:249: ArrowPar

C:\Users\Jonghyeon\Anaconda3\lib\site-packages\arrow\factory.py:249: ArrowParseWarning: The .get() parsing method with a format string will parse more strictly in version 0.15.0.See https://github.com/crsmithdev/arrow/issues/612 for more details.
  ArrowParseWarning,
Sampling:  40%|█████████████████████████████▏                                           | 4/10 [00:02<00:03,  1.99it/s]C:\Users\Jonghyeon\Anaconda3\lib\site-packages\arrow\factory.py:249: ArrowParseWarning: The .get() parsing method with a format string will parse more strictly in version 0.15.0.See https://github.com/crsmithdev/arrow/issues/612 for more details.
  ArrowParseWarning,
C:\Users\Jonghyeon\Anaconda3\lib\site-packages\arrow\factory.py:249: ArrowParseWarning: The .get() parsing method with a format string will parse more strictly in version 0.15.0.See https://github.com/crsmithdev/arrow/issues/612 for more details.
  ArrowParseWarning,
C:\Users\Jonghyeon\Anaconda3\lib\site-packages\arrow\factory.py:249: ArrowPar

Train on 4500 samples, validate on 500 samples
Epoch 1/50
4500/4500 [==============================] - ETA: 3s - loss: 0.250 - ETA: 0s - loss: 0.249 - ETA: 0s - loss: 0.249 - ETA: 0s - loss: 0.249 - 1s 178us/step - loss: 0.2486 - val_loss: 0.2472
Epoch 2/50
4500/4500 [==============================] - ETA: 0s - loss: 0.246 - ETA: 0s - loss: 0.246 - ETA: 0s - loss: 0.245 - ETA: 0s - loss: 0.245 - 0s 92us/step - loss: 0.2446 - val_loss: 0.2432
Epoch 3/50
4500/4500 [==============================] - ETA: 0s - loss: 0.241 - ETA: 0s - loss: 0.241 - ETA: 0s - loss: 0.240 - ETA: 0s - loss: 0.239 - 0s 98us/step - loss: 0.2387 - val_loss: 0.2366
Epoch 4/50
4500/4500 [==============================] - ETA: 0s - loss: 0.234 - ETA: 0s - loss: 0.233 - ETA: 0s - loss: 0.232 - ETA: 0s - loss: 0.231 - 0s 94us/step - loss: 0.2298 - val_loss: 0.2259
Epoch 5/50
4500/4500 [==============================] - ETA: 0s - loss: 0.223 - ETA: 0s - loss: 0.221 - ETA: 0s - loss: 0.219 - ETA: 0s - loss: 0.218 - 0s 9

Epoch 39/50
4500/4500 [==============================] - ETA: 0s - loss: 0.004 - ETA: 0s - loss: 0.004 - ETA: 0s - loss: 0.004 - ETA: 0s - loss: 0.004 - ETA: 0s - loss: 0.004 - 0s 99us/step - loss: 0.0041 - val_loss: 0.0038
Epoch 40/50
4500/4500 [==============================] - ETA: 0s - loss: 0.004 - ETA: 0s - loss: 0.004 - ETA: 0s - loss: 0.004 - ETA: 0s - loss: 0.004 - ETA: 0s - loss: 0.004 - 0s 94us/step - loss: 0.0040 - val_loss: 0.0038
Epoch 41/50
4500/4500 [==============================] - ETA: 0s - loss: 0.004 - ETA: 0s - loss: 0.004 - ETA: 0s - loss: 0.004 - ETA: 0s - loss: 0.004 - 0s 92us/step - loss: 0.0040 - val_loss: 0.0038
Epoch 42/50
4500/4500 [==============================] - ETA: 0s - loss: 0.003 - ETA: 0s - loss: 0.004 - ETA: 0s - loss: 0.003 - ETA: 0s - loss: 0.004 - 0s 96us/step - loss: 0.0040 - val_loss: 0.0038
Epoch 43/50
4500/4500 [==============================] - ETA: 0s - loss: 0.003 - ETA: 0s - loss: 0.003 - ETA: 0s - loss: 0.003 - ETA: 0s - loss: 0.003 -

C:\Users\Jonghyeon\Anaconda3\lib\site-packages\arrow\factory.py:249: ArrowParseWarning: The .get() parsing method with a format string will parse more strictly in version 0.15.0.See https://github.com/crsmithdev/arrow/issues/612 for more details.
  ArrowParseWarning,
C:\Users\Jonghyeon\Anaconda3\lib\site-packages\arrow\factory.py:249: ArrowParseWarning: The .get() parsing method with a format string will parse more strictly in version 0.15.0.See https://github.com/crsmithdev/arrow/issues/612 for more details.
  ArrowParseWarning,
C:\Users\Jonghyeon\Anaconda3\lib\site-packages\arrow\factory.py:249: ArrowParseWarning: The .get() parsing method with a format string will parse more strictly in version 0.15.0.See https://github.com/crsmithdev/arrow/issues/612 for more details.
  ArrowParseWarning,
DAE:  10%|███████▊                                                                      | 1/10 [00:30<04:34, 30.45s/it]

Train on 4500 samples, validate on 500 samples
Epoch 1/50
4500/4500 [==============================] - ETA: 2s - loss: 0.250 - ETA: 0s - loss: 0.249 - ETA: 0s - loss: 0.249 - ETA: 0s - loss: 0.249 - 1s 164us/step - loss: 0.2488 - val_loss: 0.2475
Epoch 2/50
4500/4500 [==============================] - ETA: 0s - loss: 0.247 - ETA: 0s - loss: 0.246 - ETA: 0s - loss: 0.246 - ETA: 0s - loss: 0.245 - 0s 94us/step - loss: 0.2452 - val_loss: 0.2439
Epoch 3/50
4500/4500 [==============================] - ETA: 0s - loss: 0.242 - ETA: 0s - loss: 0.242 - ETA: 0s - loss: 0.241 - ETA: 0s - loss: 0.240 - 0s 95us/step - loss: 0.2400 - val_loss: 0.2382
Epoch 4/50
4500/4500 [==============================] - ETA: 0s - loss: 0.236 - ETA: 0s - loss: 0.235 - ETA: 0s - loss: 0.234 - ETA: 0s - loss: 0.233 - ETA: 0s - loss: 0.233 - 0s 97us/step - loss: 0.2319 - val_loss: 0.2288
Epoch 5/50
4500/4500 [==============================] - ETA: 0s - loss: 0.226 - ETA: 0s - loss: 0.224 - ETA: 0s - loss: 0.223 - ETA:

In [3]:
datasets = sorted([e.name for e in get_event_log_files() if e.p == 0.3])
datasets

['huge-0.3-1',
 'huge-0.3-2',
 'large-0.3-1',
 'large-0.3-2',
 'medium-0.3-1',
 'medium-0.3-2',
 'small-0.3-1',
 'small-0.3-2',
 'wide-0.3-1',
 'wide-0.3-2']

In [7]:
datasets = sorted([e.name for e in get_event_log_files() if e.p == 0.3])
ads = [
    #dict(ad=RandomAnomalyDetector),
    #dict(ad=TStidePlus, ad_kwargs=dict(k=2)),
    #dict(ad=OneClassSVM),
    #dict(ad=LikelihoodPlusAnomalyDetector),
    #dict(ad=BoehmerLikelihoodAnomalyDetector),
    dict(ad=NaiveAnomalyDetector),
    #dict(ad=NaivePlusAnomalyDetector),
    #dict(ad=SamplingAnomalyDetector),
    #dict(ad=DAE, fit_kwargs=dict(epochs=50, batch_size=500)),
    #dict(ad=BINetv1, fit_kwargs=dict(epochs=20, batch_size=500)),
    #dict(ad=BINetv2, fit_kwargs=dict(epochs=20, batch_size=500)),
    #dict(ad=BINetv3, fit_kwargs=dict(epochs=20, batch_size=500))
]
for ad in ads:
    for d in tqdm(datasets, desc=ad['ad'].name):
        dataset=Dataset(d)
        dataset.id
        


Naive:   0%|                                                                                     | 0/1 [00:00<?, ?it/s]

AttributeError: 'Dataset' object has no attribute 'id'